In [ ]:
import os
import time
import open3d
from open3d import visualization
import cv2
import numpy as np
from pathlib import Path

In [ ]:
color_groups = {'red_rgb': [255,0,0], 'green_rgb': [0,128,0], 'brown_rgb': [165,42,42]}

In [ ]:
joint_grouped = {
    'center_joints': ['Hips','Spine','Spine1','Spine2','Neck','Head'],
    'right_upper_joints': ['RightShoulder','RightArm','RightForeArm','RightHand'],
    'right_lower_joints': ['RightUpLeg','RightLeg','RightFoot','RightToeBase'],
    'left_upper_joints': ['LeftShoulder','LeftArm','LeftForeArm','LeftHand'],
    'left_lower_joints': ['LeftUpLeg','LeftLeg','LeftFoot','LeftToeBase'],
}

In [ ]:
joint_colors = dict()

for joint_group_key, joint_list in joint_grouped.items():
    if joint_group_key is 'center_joints':
        for joints in joint_list:
            pass
    elif 'right' in joint_group_key:
        for joints in joint_list:
            pass
    elif 'left' in joint_group_key:
        for joints in joint_list:
            pass

In [ ]:
mdm_result_path = Path("/home/notingcode/Projects/3d/my_own/")

npy_filepaths = mdm_result_path.glob("*.npy")

all_motion_data = dict()

for npy_fp in npy_filepaths:
    motion_data = np.load(npy_fp, allow_pickle=True)
    
    for key, val in motion_data.item().items():
        if all_motion_data.get(key, None) is None:
            all_motion_data[key] = list()
        
        all_motion_data[key].append(val)
        
        print(f"key: {key} | val type: {type(val)}")
    
    print(f"\n{npy_fp} read")

In [ ]:
motion_descriptions_used = set()

action_texts_list = all_motion_data['text']

for action_texts in action_texts_list:
    for action in action_texts:
        motion_descriptions_used.add(action)

In [ ]:
motion_capture = all_motion_data['motion'][0][0][:,:,0]
print(motion_capture.shape)

In [ ]:
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(motion_capture)

In [ ]:
vis = visualization.Visualizer()
vis.create_window()
vc = vis.get_view_control()

pcd = open3d.geometry.PointCloud()
entire_motion = all_motion_data['motion'][0][20,:,:,:]

for frame_idx in range(entire_motion.shape[2]):
    pcd.clear()
    pcd.points = open3d.utility.Vector3dVector(entire_motion[:,:,frame_idx])
    if frame_idx == 0:
        vis.add_geometry(pcd)
    else:
        vis.update_geometry(pcd)
    center = pcd.get_center()
    vc.set_lookat(center)
    vc.set_front([center[0],center[1],center[2]])
    vc.set_up([1,0,0])
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()